In [1]:
import os
from urllib.parse import urlencode, unquote
import requests
import time
from datetime import datetime
import bot_settings as bs
import telepot

In [2]:
def get_file_name(url):
    file_name = url.split('&')[1][9:]
    return file_name


def download_file(public_key):
    """Загрузка файла и возврат имени загруженного файла"""
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    # Загружаем файл и сохраняем его
    download_response = requests.get(download_url)
    with open(os.path.join('data', unquote(get_file_name(download_url))), 'wb') as f:
        file_name = unquote(get_file_name(download_url))
        f.write(download_response.content)
        print('{}: загружен файл {}'.format(datetime.today(), file_name))
    
    return file_name


def calc_days_from_start(start_day=bs.start_day, *m_time):
    """Подсчет количества дней с начала марафона"""
    if m_time:
        structTime = time.localtime(*m_time)
    else:
        structTime = time.localtime()

    days_from_start = datetime(*structTime[:6]).date() - datetime.strptime(start_day, '%d.%m.%Y').date()
    days_from_start = days_from_start.days
    days_from_start
    print('{}: день марафона: {}'.format(datetime.today(), days_from_start + 1))
    return days_from_start


def send_message_to_tlg(file):
    """Отправка сообщений в телеграмм"""
    with open(os.path.join('data', file), 'r') as txt:
        bot.sendMessage(bs.chat_id, txt.read())

        
def send_video_to_tlg(file):
    """Отправка видео в телеграмм"""
    with open(os.path.join('data', file), 'rb') as mov:
        bot.sendVideo(bs.chat_id, mov, width=20, height=11)
        
        
def send_doc_to_tlg(file):
    """Отправка фалов в телеграмм"""
    with open(os.path.join('data', file), 'rb') as doc:
        bot.sendDocument(bs.chat_id, doc)

        
def send_data_to_tlg(file_name):
    file_extension = file_name.split('.')[1] # расширение файла
    if file_extension == 'txt':
        send_message_to_tlg(file_name) # закомментировать для тестирования
        print('{}: send text {}'.format(datetime.today(), file_name))
    elif file_extension == 'mov':
        send_video_to_tlg(file_name) # закомментировать для тестирования
        print('{}: send video {}'.format(datetime.today(), file_name))
    else:
        send_doc_to_tlg(file_name) # закомментировать для тестирования
        print('{}: send doc {}'.format(datetime.today(), file_name))

        
def check_start_marathon(start_day=bs.start_day, send_hour=bs.send_hour, *m_time):
    """Проверка начала марафона"""
    if m_time:
        structTime = time.localtime(*m_time)
    else:
        structTime = time.localtime()
        
    marathon_started = (datetime(*structTime[:6]).date() >= datetime.strptime(start_day, '%d.%m.%Y').date()) and \
        (time.localtime().tm_hour >= send_hour)
    return marathon_started

In [3]:
# Задание начальной даты для тестирования

# t = datetime(2020, 10, 8, 0, 0, 0) # раскомментировать для тестирования
# my_time = (t-datetime(1970,1,1)).total_seconds()# раскомментировать для тестирования
# my_time # раскомментировать для тестирования

In [4]:
# Проверка данных

total_days = datetime.strptime(bs.finish_day, '%d.%m.%Y').date() - datetime.strptime(bs.start_day, '%d.%m.%Y').date()
total_days = total_days.days
print('Общая продолжительность марафона, дней: {}'.format(total_days + 1))


if total_days + 3 != len(bs.links):
    print('Количество дней марафона не равно количеству дней в конфигурационном файле!')
else:
    print('Данные в порядке!')

Общая продолжительность марафона, дней: 4
Данные в порядке!


In [5]:
# Создание экземпляра бота
bot = telepot.Bot(bs.token)

# Создание папки data
if not os.path.exists('data'):
    os.makedirs('data')

files_list = [] # Список файлов на отправку

if not check_start_marathon(): # закомментировать для тестирования
# if not check_start_marathon(bs.start_day, bs.send_hour, my_time): # раскомментировать для тестирования
    # Загрузка файлов с диска нулевого дня
    print('{}: марафон еще не стартовал, загружаю файлы\n'.format(datetime.today()))
    for link in bs.links[0]:
        print('{}: {}'.format(datetime.today(), link))
        files_list.append(download_file(link))
        
    print('\n{}: файлы нулевого дня загружены'.format(datetime.today()))

    
    # Отправка стартовых сообщений - день 0
    print('\n{}: начинаю отправку сообщений нулевого дня\n'.format(datetime.today()))
    for file_name in files_list:
        send_data_to_tlg(file_name)
    
    print('\n{}: сообщения нулевого дня отправлены'.format(datetime.today()))
else:
    print('\n{}: марафон уже стартовал, сообщения нулевого дня не отправлены!'.format(datetime.today()))

files_list = []

2020-10-09 13:21:53.780716: марафон еще не стартовал, загружаю файлы

2020-10-09 13:21:53.784397: https://yadi.sk/d/pN4VURbpttmJnw
2020-10-09 13:21:55.269668: загружен файл day_0_msg_1.txt
2020-10-09 13:21:55.270967: https://yadi.sk/i/xCbt-CkKMutgIA
2020-10-09 13:22:04.346141: загружен файл day_0_msg_2.mov
2020-10-09 13:22:04.363277: https://yadi.sk/d/K3d3uroPcC78OA
2020-10-09 13:22:05.508842: загружен файл day_0_msg_3.txt
2020-10-09 13:22:05.510584: https://yadi.sk/i/BVn4QqGrroAEDQ
2020-10-09 13:22:07.552025: загружен файл day_0_msg_4_Стань_лучшей_версией_себя.pdf

2020-10-09 13:22:07.554233: файлы нулевого дня загружены

2020-10-09 13:22:07.554499: начинаю отправку сообщений нулевого дня

2020-10-09 13:22:08.256058: send text day_0_msg_1.txt
2020-10-09 13:22:17.742408: send video day_0_msg_2.mov
2020-10-09 13:22:17.935419: send text day_0_msg_3.txt
2020-10-09 13:22:20.628958: send doc day_0_msg_4_Стань_лучшей_версией_себя.pdf

2020-10-09 13:22:20.629916: сообщения нулевого дня отправ

In [7]:
prev_day = 99

prev_hour = 99

file_name = ''

readiness = False # флаг работы бота (False - бот не работает)

while total_days + 3 == len(bs.links):    
    
    # уведомление владельца о запуске бота
    if not readiness:
        print('{}: Бот запущен.'.format(datetime.today()))
        requests.get('https://api.telegram.org/bot{}/sendMessage'.format(bs.token), params=dict(
            chat_id=bs.owner_id,
            text='Бот запущен.'))
        readiness = True
    

    hour = time.localtime().tm_hour # закомментировать для тестирования
#     hour = time.localtime(my_time).tm_hour # раскомментировать для тестирования
    if (hour != prev_hour):
        prev_hour = hour
        
        marathon_day = calc_days_from_start() + 1 # закомментировать для тестирования
#         marathon_day = calc_days_from_start(bs.start_day, my_time) + 1  # раскомментировать для тестирования
    
        print('marathon_day=', marathon_day, ', hour=', hour, sep='')
    
        if (marathon_day < 1):
            print('\n{}: марафон еще не стартовал'.format(datetime.today()))
    
        else:
            # Загрузка файлов с диска текущего дня за час до отправки
            if hour == bs.send_hour - 2:
                print('{}: загружаю файлы\n'.format(datetime.today()))
                for link in bs.links[marathon_day]:
                    print('{}: {}'.format(datetime.today(), link))
                    files_list.append(download_file(link))

                print('\n{}: файлы загружены'.format(datetime.today()))

                # Отправка сообщения о готовности бота отправлять сообщения в чат
                print(datetime.today().time(), 'Файлы загружены. Бот готов отправлять сообщения в чат.')
                requests.get('https://api.telegram.org/bot{}/sendMessage'.format(bs.token), params=dict(
                    chat_id=bs.owner_id,
                    text='Файлы загружены. Бот готов отправлять сообщения в чат.'))  
            
            
            # Отправка сообщений за час до назначенного времени
            if (hour == bs.send_hour - 1) and (marathon_day != 1) and (marathon_day != len(bs.links) - 1):
                print('\n{}: отправляю первое сообщение\n'.format(datetime.today()))
                for file_name in files_list:
                    if 'msg_1.txt' in file_name:
                        send_data_to_tlg(file_name)
                print('\n{}: первое сообщение отправлено\n'.format(datetime.today()))
                del files_list[0]

            # Отправка сообщений в назначенное время
            if hour == bs.send_hour:
                print('\n{}: начинаю отправку сообщений\n'.format(datetime.today()))
                for file_name in files_list:
                    send_data_to_tlg(file_name)
                print('\n{}: сообщения отправлены\n'.format(datetime.today()))
                
                if file_name == 'day_П_msg_1.txt':
                    print('\n{}: Congrats! Marathon is over.'.format(datetime.today()))
                
                files_list = []

        print(files_list)    
                
        if marathon_day >= total_days + 3:
            break
            
            
        # Для того, чтобы не падало соединение в каждом цикле запрашивается информация о боте и программа останавливается на 60 секунд
        bot.getMe()
        time.sleep(60)
            
            
#     my_time += 0.05 # раскомментировать для тестирования

2020-10-09 13:22:20.752917: Бот запущен.
2020-10-09 13:22:26.179631: день марафона: 0
marathon_day=0, hour=3

2020-10-09 13:22:26.181974: марафон еще не стартовал
[]
2020-10-09 13:22:26.625150: день марафона: 0
marathon_day=0, hour=4

2020-10-09 13:22:26.626027: марафон еще не стартовал
[]
2020-10-09 13:22:27.028658: день марафона: 0
marathon_day=0, hour=5

2020-10-09 13:22:27.029378: марафон еще не стартовал
[]
2020-10-09 13:22:27.431325: день марафона: 0
marathon_day=0, hour=6

2020-10-09 13:22:27.432170: марафон еще не стартовал
[]
2020-10-09 13:22:27.859985: день марафона: 0
marathon_day=0, hour=7

2020-10-09 13:22:27.860675: марафон еще не стартовал
[]
2020-10-09 13:22:28.253702: день марафона: 0
marathon_day=0, hour=8

2020-10-09 13:22:28.254950: марафон еще не стартовал
[]
2020-10-09 13:22:28.665194: день марафона: 0
marathon_day=0, hour=9

2020-10-09 13:22:28.665683: марафон еще не стартовал
[]
2020-10-09 13:22:29.058410: день марафона: 0
marathon_day=0, hour=10

2020-10-09 13:

2020-10-09 13:40:13.412508: загружен файл day_2_msg_7_part_5-Тест_на_количество_энергии.aac
2020-10-09 13:40:13.433676: https://yadi.sk/d/T7GwBTsvrd3hew
2020-10-09 13:40:32.278761: загружен файл day_2_msg_8_part_6-Техника_разговор_с_ребёнком.m4a
2020-10-09 13:40:32.293931: https://yadi.sk/d/f6_mtIswChBKfQ
2020-10-09 13:40:54.329695: загружен файл day_2_msg_9_part_7-Денежная_дорожка.m4a
2020-10-09 13:40:54.341563: https://yadi.sk/d/WIYLj0AMDCVsoQ
2020-10-09 13:40:56.152517: загружен файл day_2_msg_10.txt

2020-10-09 13:40:56.154789: файлы загружены
13:40:56.155725 Файлы загружены. Бот готов отправлять сообщения в чат.
['day_2_msg_1.txt', 'day_2_msg_2.txt', 'day_2_msg_3_part_1.mp3', 'day_2_msg_4_part_2.aac', 'day_2_msg_5_part_3.aac', 'day_2_msg_6_part_4.m4a', 'day_2_msg_7_part_5-Тест_на_количество_энергии.aac', 'day_2_msg_8_part_6-Техника_разговор_с_ребёнком.m4a', 'day_2_msg_9_part_7-Денежная_дорожка.m4a', 'day_2_msg_10.txt']
2020-10-09 13:40:57.174287: день марафона: 2
marathon_day=2, h

2020-10-09 13:52:36.638124: день марафона: 4
marathon_day=4, hour=3
[]
2020-10-09 13:52:37.123522: день марафона: 4
marathon_day=4, hour=4
[]
2020-10-09 13:52:37.517503: день марафона: 4
marathon_day=4, hour=5
[]
2020-10-09 13:52:37.928578: день марафона: 4
marathon_day=4, hour=6
[]
2020-10-09 13:52:38.484731: день марафона: 4
marathon_day=4, hour=7
[]
2020-10-09 13:52:38.901170: день марафона: 4
marathon_day=4, hour=8
2020-10-09 13:52:38.901988: загружаю файлы

2020-10-09 13:52:38.902079: https://yadi.sk/d/ILjV3oCY-zgjXw
2020-10-09 13:52:40.458008: загружен файл day_4_msg_1.txt
2020-10-09 13:52:40.459078: https://yadi.sk/d/IG-x0HCX7LJeXg
2020-10-09 13:52:42.035666: загружен файл day_4_msg_2.txt
2020-10-09 13:52:42.037863: https://yadi.sk/d/DOaB4Ale1EgFBg
2020-10-09 13:53:53.358713: загружен файл day_4_msg_3_part_1.mp3
2020-10-09 13:53:53.401898: https://yadi.sk/d/OA-572qlP6Hj_A
2020-10-09 13:54:33.069032: загружен файл day_4_msg_4_part_2.m4a
2020-10-09 13:54:33.126918: https://yadi.sk